# Spill the Beans: RAG Privacy Attack - Colab Setup

This notebook reproduces Table 1 from the paper "Follow My Instruction and Spill the Beans" (ICLR 2025).

**GPU Required**: 
- **7B models**: Runtime > Change runtime type > T4 GPU
- **13B models**: Runtime > Change runtime type > A100 GPU (Colab Pro required)

## 1. Clone Repository

In [ ]:
!git clone https://github.com/aamangeldi/spill-the-beans.git
%cd spill-the-beans

## 2. Install Dependencies

In [ ]:
!pip install -e .

## 2b. Authenticate with HuggingFace (Required for Llama2)

**Important**: Llama2 models require HuggingFace authentication and access approval.

**Option 1: Use Colab Secrets (Recommended)**
1. Get your token at: https://huggingface.co/settings/tokens
2. In Colab: Click 🔑 (Key icon) in left sidebar → "Add new secret"
3. Name: `HF_TOKEN`, Value: paste your token
4. Enable notebook access (toggle switch)
5. Run the cell below (auto-detects secret)

**Option 2: Manual Login**
If you don't have a secret set up, you'll be prompted to enter your token.

**Then request Llama2 access (one-time, usually instant):**
- Visit [Llama2-7B](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf)
- Fill out the access request form (name, email, affiliation, intended use)
- Click "Submit" - approval is usually instant
- This grants access to **all** Llama2 models (7B, 13B, etc.)

**Note**: Other models (Mistral, SOLAR, Vicuna, Mixtral, WizardLM) work without authentication.

In [ ]:
from huggingface_hub import notebook_login
import os

# Try to use Colab secret first (more convenient)
try:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        os.environ['HF_TOKEN'] = hf_token
        print("✓ Using HF_TOKEN from Colab secrets")
    else:
        raise KeyError("HF_TOKEN not found")
except (ImportError, KeyError):
    # Fall back to interactive login
    print("No HF_TOKEN secret found. Please enter your token manually:")
    notebook_login()

## 3. Verify GPU Availability

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 4. Quick Test (10 samples)

Test with Mistral-7B to verify everything works (~5 minutes).

**Note**: This test doesn't require HuggingFace authentication (Mistral is public).

In [ ]:
!python src/main.py --models mistral-7b --num-samples 10

## 5. Reproduce Table 1

Run all models from the paper to reproduce Table 1.

**Prerequisites**: 
- ✅ Complete Step 2b (HuggingFace authentication) for Llama2 models
- ✅ A100 GPU required for 13B models and Mixtral-8x7b

**Total time**: ~4-5 hours on A100 GPU

**Models tested**:
- **7B**: Llama2-7B, Mistral-7B
- **10.7B**: SOLAR-10.7B  
- **13B**: Llama2-13B, Vicuna-13B, WizardLM-13B
- **8x7B**: Mixtral-8x7B (with automatic 4-bit quantization)

**Note**: Using 100 samples instead of 230 for faster runtime while maintaining statistical significance.

**Mixtral-8x7b**: Automatically uses 4-bit quantization to fit in A100 (40GB VRAM). Results may differ slightly from paper's fp16 implementation.

In [ ]:
!python src/main.py --models llama2-7b mistral-7b solar-10.7b llama2-13b vicuna-13b mixtral-8x7b wizardlm-13b --num-samples 100

## 6. View Final Results

Display all experiment results in a summary table.

In [ ]:
!python src/view_results.py

## 7. Download Results

Download all experiment results as a zip file.

In [ ]:
from google.colab import files

# Zip all results
!zip -r results.zip results/

# Download
files.download('results.zip')